In [1]:
import numpy as np
import pandas as pd
import keras
from sklearn import preprocessing
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [2]:
import matplotlib.pyplot as plt
import os, os.path, sys
import copy

In [5]:
path = r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\podaci\0k\X"

In [6]:
dir = os.listdir(path)

In [7]:
file_names = []
for file in dir:
    file_names.append(file)

In [8]:
test_file_names = []
train_file_names = copy.copy(file_names)
file_indx = np.sort(np.random.randint(1,30,5))[::-1]

for ind in file_indx:
    test_file_names.append(file_names[ind])
    train_file_names.pop(ind)

Potencijalni time_stepovi su (1, 2, 4, 5, 8, 10, 20, 25, 40, 50, 61, 100), kako bi bili deljivi sa 61000

In [9]:
train_ = np.concatenate([pd.read_csv(path + "\\" + file, header=None)[:61000].astype('int')
                              for file in train_file_names], axis=0)
test_ = np.concatenate([pd.read_csv(path + "\\" + file, header=None)[:61000].astype('int')
                              for file in test_file_names], axis=0)

In [244]:
scaler = preprocessing.MinMaxScaler(feature_range=(0,100))

In [245]:
scaler.fit(train_)

MinMaxScaler(feature_range=(0, 100))

In [246]:
train_ = scaler.transform(train_)
test_ = scaler.transform(test_)

best step = 20

In [93]:
def create_timeseries_split(data, time_steps):

  # Get the number of samples in the data
  num_samples = len(data) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data[i+time_steps, 0]

  return x_train, y_train

In [247]:
time_steps = 2

In [248]:
train_X, train_y = create_timeseries_split(train_, time_steps)

In [249]:
test_X, test_y = create_timeseries_split(test_, time_steps)

In [106]:
np.random.seed(1234)

In [107]:
input_shape = (time_steps,1)

In [268]:
model = Sequential()
model.add(LSTM(units = 16, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
model.add(Dense(units = 16, activation=keras.layers.LeakyReLU()))
model.add(Dense(units = 16, activation=keras.layers.LeakyReLU()))
model.add(Dense(units = 16, activation=keras.layers.LeakyReLU()))
model.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])

In [269]:
history = model.fit(train_X, train_y, epochs=20, batch_size = 32, verbose = 2)

Epoch 1/20

47657/47657 - 92s - loss: 5.0176 - accuracy: 1.3115e-06 - 92s/epoch - 2ms/step
Epoch 2/20
47657/47657 - 99s - loss: 0.0070 - accuracy: 1.3115e-06 - 99s/epoch - 2ms/step
Epoch 3/20
47657/47657 - 96s - loss: 0.0057 - accuracy: 1.3115e-06 - 96s/epoch - 2ms/step
Epoch 4/20
47657/47657 - 97s - loss: 0.0053 - accuracy: 1.3115e-06 - 97s/epoch - 2ms/step
Epoch 5/20
47657/47657 - 98s - loss: 0.0052 - accuracy: 1.3115e-06 - 98s/epoch - 2ms/step
Epoch 6/20
47657/47657 - 85s - loss: 0.0046 - accuracy: 1.3115e-06 - 85s/epoch - 2ms/step
Epoch 7/20
47657/47657 - 74s - loss: 0.0048 - accuracy: 1.3115e-06 - 74s/epoch - 2ms/step
Epoch 8/20
47657/47657 - 74s - loss: 0.0042 - accuracy: 1.3115e-06 - 74s/epoch - 2ms/step
Epoch 9/20
47657/47657 - 90s - loss: 0.0041 - accuracy: 1.3115e-06 - 90s/epoch - 2ms/step
Epoch 10/20
47657/47657 - 92s - loss: 0.0039 - accuracy: 1.3115e-06 - 92s/epoch - 2ms/step
Epoch 11/20
47657/47657 - 92s - loss: 0.0035 - accuracy: 1.3115e-06 - 92s/epoch - 2ms/step
Epoch 1

In [270]:
train_scores = model.evaluate(test_X, test_y)

9532/9532 [==============================] - 26s 2ms/step - loss: 2.2311e-04 - accuracy: 0.0000e+00


In [271]:
train_predict = model.predict(train_X)
test_predict = model.predict(test_X)

9532/9532 [==============================] - 15s 2ms/step


In [272]:
n = 61000
test_size = len(test_file_names)
avg_koord = []
for i in range(n):
    avg_koord.append(sum(test_[i] + test_[n+i] + test_[2*n + i] + test_[3*n + i] + test_[4*n + i])/test_size)
    

In [273]:
num_correct_train = 0
for i in range(n):
    if train_predict[i][0] <= (avg_koord[i]+avg_koord[i]*0.0003):
        if train_predict[i][0] >= (avg_koord[i]-avg_koord[i]*0.0003):
            num_correct_train += 1
num_correct_train

15934

In [274]:
num_correct_test = 0
for i in range(n):
    if test_predict[i][0] <= (avg_koord[i]+avg_koord[i]*0.0003):
        if test_predict[i][0] >= (avg_koord[i]-avg_koord[i]*0.0003):
            num_correct_test += 1
num_correct_test

15544

In [275]:
for i in range(n):
    if train_predict[i][0] <= (avg_koord[i]+avg_koord[i]*0.0003):
        if train_predict[i][0] >= (avg_koord[i]-avg_koord[i]*0.0003):
            print((np.round((avg_koord[i]-avg_koord[i]*0.0003),decimals=4), np.round(train_predict[i][0], decimals=4), np.round((avg_koord[i]+avg_koord[i]*0.0003),decimals=4)), 'index:', i)

(5.515, 5.5162, 5.5183) index: 275
(5.5196, 5.5224, 5.5229) index: 276
(5.578, 5.5803, 5.5814) index: 293
(5.5718, 5.575, 5.5752) index: 295
(5.5776, 5.5788, 5.5809) index: 300
(5.5755, 5.5767, 5.5788) index: 301
(5.5737, 5.5752, 5.577) index: 302
(5.5773, 5.5773, 5.5806) index: 306
(5.5755, 5.5765, 5.5788) index: 308
(5.5747, 5.5767, 5.578) index: 312
(5.5733, 5.5754, 5.5766) index: 318
(5.5672, 5.5693, 5.5705) index: 4304
(5.5604, 5.5609, 5.5637) index: 4306
(5.5528, 5.5537, 5.5562) index: 4310
(5.4429, 5.4436, 5.4462) index: 4326
(5.4382, 5.4406, 5.4415) index: 4327
(99.0229, 99.0785, 99.0823) index: 8363
(99.1959, 99.2547, 99.2554) index: 8371
(99.2144, 99.2676, 99.274) index: 8372
(99.2299, 99.2845, 99.2894) index: 8373
(99.3513, 99.4072, 99.411) index: 8379
(99.3668, 99.423, 99.4264) index: 8380
(99.4545, 99.5139, 99.5142) index: 8385
(99.4701, 99.5263, 99.5298) index: 8386
(99.4845, 99.5405, 99.5442) index: 8387
(99.5004, 99.5574, 99.5602) index: 8388
(99.5693, 99.6272, 99.629) 

(94.3947, 94.4365, 94.4514) index: 11627
(94.3947, 94.4365, 94.4514) index: 11628
(94.3947, 94.4365, 94.4514) index: 11629
(94.3947, 94.4365, 94.4514) index: 11630
(94.3947, 94.4365, 94.4514) index: 11631
(94.3947, 94.4365, 94.4514) index: 11632
(94.3947, 94.4365, 94.4514) index: 11633
(94.3947, 94.4365, 94.4514) index: 11634
(94.3947, 94.4365, 94.4514) index: 11635
(94.3947, 94.4365, 94.4514) index: 11636
(94.3947, 94.4365, 94.4514) index: 11637
(94.3947, 94.4365, 94.4514) index: 11638
(94.3947, 94.4365, 94.4514) index: 11639
(94.3947, 94.4365, 94.4514) index: 11640
(94.3947, 94.4365, 94.4514) index: 11641
(94.3947, 94.4365, 94.4514) index: 11642
(94.3947, 94.4365, 94.4514) index: 11643
(94.3947, 94.4365, 94.4514) index: 11644
(94.3947, 94.4365, 94.4514) index: 11645
(94.3947, 94.4365, 94.4514) index: 11646
(94.3947, 94.4365, 94.4514) index: 11647
(94.3947, 94.4365, 94.4514) index: 11648
(94.3947, 94.4365, 94.4514) index: 11649
(94.3947, 94.4365, 94.4514) index: 11650
(94.3947, 94.436

(94.3963, 94.4354, 94.453) index: 22376
(94.3963, 94.4354, 94.453) index: 22377
(94.3963, 94.4354, 94.453) index: 22378
(94.3963, 94.4354, 94.453) index: 22379
(94.3963, 94.4354, 94.453) index: 22380
(94.3963, 94.4354, 94.453) index: 22381
(94.3963, 94.4354, 94.453) index: 22382
(94.3963, 94.4354, 94.453) index: 22383
(94.3963, 94.4354, 94.453) index: 22384
(94.3963, 94.4354, 94.453) index: 22385
(94.3963, 94.4354, 94.453) index: 22386
(94.3963, 94.4354, 94.453) index: 22387
(94.3963, 94.4354, 94.453) index: 22388
(94.3963, 94.4354, 94.453) index: 22389
(94.3963, 94.4354, 94.453) index: 22390
(94.3963, 94.4354, 94.453) index: 22391
(94.3963, 94.4354, 94.453) index: 22392
(94.3963, 94.4354, 94.453) index: 22393
(94.3963, 94.4354, 94.453) index: 22394
(94.3963, 94.4354, 94.453) index: 22395
(94.3963, 94.4354, 94.453) index: 22396
(94.3963, 94.4354, 94.453) index: 22397
(94.3963, 94.4354, 94.453) index: 22398
(94.3963, 94.4354, 94.453) index: 22399
(94.3963, 94.4354, 94.453) index: 22400


(88.6466, 88.6504, 88.6998) index: 48692
(88.6192, 88.6232, 88.6724) index: 48693
(88.5916, 88.593, 88.6448) index: 48694
(88.564, 88.565, 88.6172) index: 48695
(88.5356, 88.5382, 88.5887) index: 48696
(88.5076, 88.511, 88.5607) index: 48697
(88.48, 88.4838, 88.5331) index: 48698
(88.4526, 88.4544, 88.5057) index: 48699
(88.4251, 88.427, 88.4782) index: 48700
(88.3965, 88.3992, 88.4495) index: 48701
(88.3683, 88.3722, 88.4213) index: 48702
(88.3417, 88.345, 88.3947) index: 48703
(88.314, 88.3148, 88.367) index: 48704
(88.2859, 88.2876, 88.3389) index: 48705
(88.2577, 88.2599, 88.3106) index: 48706
(88.2299, 88.2336, 88.2829) index: 48707
(88.2026, 88.2062, 88.2555) index: 48708
(88.1755, 88.1775, 88.2284) index: 48709
(88.1479, 88.1492, 88.2008) index: 48710
(88.1194, 88.1224, 88.1723) index: 48711
(88.0912, 88.0951, 88.1441) index: 48712
(88.0639, 88.0672, 88.1168) index: 48713
(88.0365, 88.0379, 88.0893) index: 48714
(88.0087, 88.0098, 88.0616) index: 48715
(87.9804, 87.9822, 88.0332

(99.9469, 99.9844, 100.0069) index: 53304
(99.9469, 99.9844, 100.0069) index: 53305
(99.9469, 99.9844, 100.0069) index: 53306
(99.9469, 99.9844, 100.0069) index: 53307
(99.9469, 99.9844, 100.0069) index: 53308
(99.9469, 99.9844, 100.0069) index: 53309
(99.9469, 99.9844, 100.0069) index: 53310
(99.9469, 99.9844, 100.0069) index: 53311
(99.9469, 99.9844, 100.0069) index: 53312
(99.9469, 99.9844, 100.0069) index: 53313
(99.9469, 99.9844, 100.0069) index: 53314
(99.9469, 99.9844, 100.0069) index: 53315
(99.9469, 99.9844, 100.0069) index: 53316
(99.9469, 99.9844, 100.0069) index: 53317
(99.9469, 99.9844, 100.0069) index: 53318
(99.9469, 99.9844, 100.0069) index: 53319
(99.9469, 99.9844, 100.0069) index: 53320
(99.9469, 99.9844, 100.0069) index: 53321
(99.9469, 99.9844, 100.0069) index: 53322
(99.9469, 99.9844, 100.0069) index: 53323
(99.9469, 99.9844, 100.0069) index: 53324
(99.9469, 99.9844, 100.0069) index: 53325
(99.9469, 99.9844, 100.0069) index: 53326
(99.9469, 99.9844, 100.0069) index

In [277]:
for i in range(n):
    print((np.round((avg_koord[i]-avg_koord[i]*0.0003),decimals=4), np.round(train_predict[i][0], decimals=4), np.round((avg_koord[i]+avg_koord[i]*0.0003),decimals=4)), 'index:',i)

(0.0189, 0.0176, 0.0189) index: 0
(0.0189, 0.0176, 0.0189) index: 1
(0.019, 0.0191, 0.019) index: 2
(0.0199, 0.021, 0.0199) index: 3
(0.022, 0.0241, 0.022) index: 4
(0.0259, 0.0291, 0.0259) index: 5
(0.0305, 0.0351, 0.0306) index: 6
(0.035, 0.04, 0.035) index: 7
(0.0379, 0.0399, 0.0379) index: 8
(0.0388, 0.0396, 0.0388) index: 9
(0.0391, 0.0403, 0.0391) index: 10
(0.0403, 0.0416, 0.0403) index: 11
(0.0426, 0.0457, 0.0427) index: 12
(0.047, 0.0587, 0.047) index: 13
(0.0536, 0.0732, 0.0537) index: 14
(0.062, 0.0809, 0.062) index: 15
(0.0695, 0.0826, 0.0695) index: 16
(0.0755, 0.0838, 0.0755) index: 17
(0.0786, 0.0841, 0.0787) index: 18
(0.0818, 0.0883, 0.0819) index: 19
(0.0868, 0.0984, 0.0869) index: 20
(0.0936, 0.1112, 0.0937) index: 21
(0.101, 0.1264, 0.1011) index: 22
(0.1124, 0.1447, 0.1124) index: 23
(0.1284, 0.1596, 0.1284) index: 24
(0.1435, 0.1587, 0.1435) index: 25
(0.1498, 0.1624, 0.1499) index: 26
(0.1544, 0.1631, 0.1545) index: 27
(0.1567, 0.1682, 0.1568) index: 28
(0.1601, 

(6.0981, 6.1509, 6.1018) index: 5014
(6.1272, 6.1811, 6.1309) index: 5015
(6.1567, 6.2087, 6.1604) index: 5016
(6.184, 6.2343, 6.1878) index: 5017
(6.2108, 6.2619, 6.2145) index: 5018
(6.2381, 6.2892, 6.2419) index: 5019
(6.2657, 6.3173, 6.2694) index: 5020
(6.2937, 6.3461, 6.2975) index: 5021
(6.3225, 6.3763, 6.3263) index: 5022
(6.3498, 6.4021, 6.3536) index: 5023
(6.3772, 6.4293, 6.3811) index: 5024
(6.4058, 6.4575, 6.4096) index: 5025
(6.4338, 6.4843, 6.4376) index: 5026
(6.4618, 6.5134, 6.4657) index: 5027
(6.4896, 6.5418, 6.4935) index: 5028
(6.5168, 6.5692, 6.5207) index: 5029
(6.5442, 6.597, 6.5482) index: 5030
(6.5721, 6.6239, 6.576) index: 5031
(6.5999, 6.6502, 6.6038) index: 5032
(6.6266, 6.6778, 6.6306) index: 5033
(6.6544, 6.7087, 6.6584) index: 5034
(6.6827, 6.7363, 6.6867) index: 5035
(6.7112, 6.7638, 6.7153) index: 5036
(6.7392, 6.7916, 6.7433) index: 5037
(6.7664, 6.8175, 6.7705) index: 5038
(6.7938, 6.8451, 6.7979) index: 5039
(6.8211, 6.8733, 6.8252) index: 5040
(6.8

(94.4262, 94.4507, 94.4828) index: 8880
(94.4161, 94.4372, 94.4727) index: 8881
(94.4076, 94.4293, 94.4643) index: 8882
(94.4023, 94.4289, 94.459) index: 8883
(94.4002, 94.4351, 94.4568) index: 8884
(94.4003, 94.4375, 94.457) index: 8885
(94.4017, 94.4381, 94.4584) index: 8886
(94.3995, 94.4347, 94.4562) index: 8887
(94.3955, 94.4264, 94.4522) index: 8888
(94.3904, 94.4213, 94.4471) index: 8889
(94.3856, 94.4202, 94.4423) index: 8890
(94.3824, 94.4217, 94.4391) index: 8891
(94.383, 94.4261, 94.4396) index: 8892
(94.3874, 94.4324, 94.4441) index: 8893
(94.3909, 94.4327, 94.4475) index: 8894
(94.3919, 94.4291, 94.4485) index: 8895
(94.3905, 94.4246, 94.4472) index: 8896
(94.3876, 94.4237, 94.4443) index: 8897
(94.3862, 94.4258, 94.4428) index: 8898
(94.3861, 94.4269, 94.4427) index: 8899
(94.3873, 94.428, 94.444) index: 8900
(94.3884, 94.4292, 94.4451) index: 8901
(94.389, 94.4288, 94.4456) index: 8902
(94.3884, 94.4273, 94.4451) index: 8903
(94.3876, 94.4276, 94.4443) index: 8904
(94.38

(94.904, 94.9546, 94.961) index: 12955
(94.9195, 94.9712, 94.9765) index: 12956
(94.931, 94.9894, 94.988) index: 12957
(94.9435, 95.0056, 95.0004) index: 12958
(94.9601, 95.023, 95.0171) index: 12959
(94.9786, 95.0388, 95.0356) index: 12960
(94.9986, 95.0548, 95.0557) index: 12961
(95.0185, 95.0762, 95.0756) index: 12962
(95.0375, 95.0984, 95.0945) index: 12963
(95.058, 95.1216, 95.1151) index: 12964
(95.0769, 95.1428, 95.134) index: 12965
(95.0958, 95.1607, 95.1529) index: 12966
(95.1156, 95.1766, 95.1727) index: 12967
(95.1348, 95.194, 95.1919) index: 12968
(95.1549, 95.2134, 95.212) index: 12969
(95.1746, 95.2338, 95.2318) index: 12970
(95.194, 95.2582, 95.2511) index: 12971
(95.2137, 95.2891, 95.2709) index: 12972
(95.2347, 95.3194, 95.2919) index: 12973
(95.2588, 95.3471, 95.316) index: 12974
(95.2836, 95.3687, 95.3407) index: 12975
(95.3076, 95.383, 95.3649) index: 12976
(95.3299, 95.3936, 95.3871) index: 12977
(95.3505, 95.4171, 95.4077) index: 12978
(95.3705, 95.4421, 95.4277) 

(10.2298, 10.1974, 10.236) index: 16627
(10.2021, 10.1684, 10.2082) index: 16628
(10.1748, 10.14, 10.1809) index: 16629
(10.147, 10.1117, 10.1531) index: 16630
(10.1186, 10.0852, 10.1247) index: 16631
(10.0907, 10.0589, 10.0968) index: 16632
(10.0635, 10.0313, 10.0695) index: 16633
(10.036, 10.0004, 10.042) index: 16634
(10.0081, 9.9737, 10.0141) index: 16635
(9.9798, 9.9459, 9.9858) index: 16636
(9.9519, 9.92, 9.9579) index: 16637
(9.9248, 9.8924, 9.9308) index: 16638
(9.8975, 9.8643, 9.9035) index: 16639
(9.8701, 9.8356, 9.876) index: 16640
(9.8419, 9.8072, 9.8478) index: 16641
(9.8139, 9.7798, 9.8198) index: 16642
(9.7866, 9.753, 9.7925) index: 16643
(9.7588, 9.7239, 9.7647) index: 16644
(9.7312, 9.6965, 9.737) index: 16645
(9.7024, 9.6687, 9.7083) index: 16646
(9.6748, 9.641, 9.6806) index: 16647
(9.6469, 9.6132, 9.6527) index: 16648
(9.6194, 9.5845, 9.6252) index: 16649
(9.5915, 9.5562, 9.5973) index: 16650
(9.5635, 9.5306, 9.5693) index: 16651
(9.5356, 9.503, 9.5413) index: 16652

...

(0.0173, 0.0165, 0.0173) index: 41220
(0.0173, 0.0165, 0.0173) index: 41221
(0.0173, 0.0165, 0.0173) index: 41222
(0.0173, 0.0165, 0.0173) index: 41223
(0.0173, 0.0165, 0.0173) index: 41224
(0.0173, 0.0165, 0.0173) index: 41225
(0.0173, 0.0165, 0.0173) index: 41226
(0.0173, 0.0165, 0.0173) index: 41227
(0.0173, 0.0165, 0.0173) index: 41228
(0.0173, 0.0165, 0.0173) index: 41229
(0.0173, 0.0165, 0.0173) index: 41230
(0.0173, 0.0165, 0.0173) index: 41231
(0.0173, 0.0165, 0.0173) index: 41232
(0.0173, 0.0165, 0.0173) index: 41233
(0.0173, 0.0165, 0.0173) index: 41234
(0.0173, 0.0165, 0.0173) index: 41235
(0.0173, 0.0165, 0.0173) index: 41236
(0.0173, 0.0165, 0.0173) index: 41237
(0.0173, 0.0165, 0.0173) index: 41238
(0.0173, 0.0165, 0.0173) index: 41239
(0.0173, 0.0165, 0.0173) index: 41240
(0.0173, 0.0165, 0.0173) index: 41241
(0.0173, 0.0165, 0.0173) index: 41242
(0.0173, 0.0165, 0.0173) index: 41243
(0.0173, 0.0165, 0.0173) index: 41244
(0.0173, 0.0165, 0.0173) index: 41245
(0.0173, 0.0

(20.8659, 20.9273, 20.8785) index: 45186
(20.8945, 20.9556, 20.907) index: 45187
(20.922, 20.9831, 20.9346) index: 45188
(20.9494, 21.0109, 20.962) index: 45189
(20.9765, 21.0378, 20.9891) index: 45190
(21.0043, 21.0667, 21.0169) index: 45191
(21.0325, 21.0942, 21.0451) index: 45192
(21.0605, 21.1228, 21.0731) index: 45193
(21.0883, 21.1494, 21.101) index: 45194
(21.1156, 21.1766, 21.1283) index: 45195
(21.1438, 21.2047, 21.1565) index: 45196
(21.1719, 21.2324, 21.1846) index: 45197
(21.1995, 21.2602, 21.2122) index: 45198
(21.2268, 21.2871, 21.2395) index: 45199
(21.2541, 21.3143, 21.2669) index: 45200
(21.2821, 21.3432, 21.2949) index: 45201
(21.3107, 21.3715, 21.3235) index: 45202
(21.3386, 21.399, 21.3514) index: 45203
(21.3665, 21.4259, 21.3793) index: 45204
(21.3937, 21.454, 21.4065) index: 45205
(21.4212, 21.4817, 21.4341) index: 45206
(21.4496, 21.5103, 21.4625) index: 45207
(21.4775, 21.5378, 21.4904) index: 45208
(21.5049, 21.5647, 21.5178) index: 45209
(21.5324, 21.5919, 21.

(78.8497, 78.8874, 78.897) index: 49245
(78.8497, 78.8874, 78.897) index: 49246
(78.8497, 78.8874, 78.897) index: 49247
(78.8497, 78.8874, 78.897) index: 49248
(78.8497, 78.8874, 78.897) index: 49249
(78.8497, 78.8874, 78.897) index: 49250
(78.8497, 78.8874, 78.897) index: 49251
(78.8497, 78.8874, 78.897) index: 49252
(78.8498, 78.8874, 78.8971) index: 49253
(78.8506, 78.8928, 78.8979) index: 49254
(78.8544, 78.8982, 78.9017) index: 49255
(78.8591, 78.9033, 78.9065) index: 49256
(78.8631, 78.9052, 78.9105) index: 49257
(78.8665, 78.9072, 78.9138) index: 49258
(78.8679, 78.9071, 78.9152) index: 49259
(78.8683, 78.9084, 78.9157) index: 49260
(78.8695, 78.9103, 78.9168) index: 49261
(78.8716, 78.9127, 78.9189) index: 49262
(78.8753, 78.9173, 78.9227) index: 49263
(78.8808, 78.9227, 78.9281) index: 49264
(78.8881, 78.93, 78.9355) index: 49265
(78.8965, 78.9382, 78.9439) index: 49266
(78.9034, 78.9444, 78.9508) index: 49267
(78.9078, 78.948, 78.9551) index: 49268
(78.9104, 78.9509, 78.9578)

(99.9469, 99.9844, 100.0069) index: 53346
(99.9469, 99.9844, 100.0069) index: 53347
(99.9469, 99.9844, 100.0069) index: 53348
(99.9469, 99.9844, 100.0069) index: 53349
(99.9469, 99.9844, 100.0069) index: 53350
(99.9469, 99.9844, 100.0069) index: 53351
(99.9469, 99.9844, 100.0069) index: 53352
(99.9469, 99.9844, 100.0069) index: 53353
(99.9469, 99.9844, 100.0069) index: 53354
(99.9469, 99.9844, 100.0069) index: 53355
(99.9469, 99.9844, 100.0069) index: 53356
(99.9469, 99.9844, 100.0069) index: 53357
(99.9469, 99.9844, 100.0069) index: 53358
(99.9469, 99.9844, 100.0069) index: 53359
(99.9469, 99.9844, 100.0069) index: 53360
(99.9469, 99.9844, 100.0069) index: 53361
(99.9469, 99.9844, 100.0069) index: 53362
(99.9469, 99.9844, 100.0069) index: 53363
(99.9469, 99.9844, 100.0069) index: 53364
(99.9469, 99.9844, 100.0069) index: 53365
(99.9469, 99.9844, 100.0069) index: 53366
(99.9469, 99.9844, 100.0069) index: 53367
(99.9469, 99.9844, 100.0069) index: 53368
(99.9469, 99.9844, 100.0069) index

(71.2158, 71.2065, 71.2585) index: 57210
(71.1873, 71.1805, 71.23) index: 57211
(71.1595, 71.1538, 71.2022) index: 57212
(71.1321, 71.1264, 71.1748) index: 57213
(71.1049, 71.096, 71.1475) index: 57214
(71.0774, 71.0681, 71.1201) index: 57215
(71.0491, 71.0406, 71.0918) index: 57216
(71.0211, 71.0144, 71.0638) index: 57217
(70.9937, 70.9877, 71.0363) index: 57218
(70.9667, 70.9587, 71.0093) index: 57219
(70.9392, 70.9295, 70.9818) index: 57220
(70.9108, 70.9022, 70.9533) index: 57221
(70.8821, 70.8744, 70.9247) index: 57222
(70.8547, 70.8474, 70.8972) index: 57223
(70.8272, 70.8194, 70.8697) index: 57224
(70.8, 70.7909, 70.8425) index: 57225
(70.7717, 70.7641, 70.8141) index: 57226
(70.7438, 70.7353, 70.7862) index: 57227
(70.7159, 70.7078, 70.7584) index: 57228
(70.6883, 70.6784, 70.7307) index: 57229
(70.6604, 70.6511, 70.7028) index: 57230
(70.6323, 70.6249, 70.6747) index: 57231
(70.6042, 70.5974, 70.6466) index: 57232
(70.5771, 70.5702, 70.6195) index: 57233
(70.5495, 70.5398, 70.